In [1]:
%run simulator.py

In [2]:
import simulator

# Read the data:

file = 'simulation_results.txt'

hgt_rates, gene_presence_absence_matrices_unfiltered = simulator.read_simulation_results(file)

# Filter out completely lost genes:

gene_presence_absence_matrices = []
for i in range(0,len(gene_presence_absence_matrices_unfiltered)):
    gene_presence_absence_matrices.append(np.array([row for row in gene_presence_absence_matrices_unfiltered[i] if not (np.all(row == 0))]))
    #gene_presence_absence_matrices.append(np.array([row for row in gene_presence_absence_matrices_unfiltered[i] if not (np.all(row == 0) or np.all(row == 1))]))

# 


In [1]:
# Packages
import gene_model
import gfs
import color_scheme
import simulator

from matplotlib import pyplot as plt
import seaborn as sns

import sbi
import numpy as np
import multiprocessing

import torch

from sbi.analysis import pairplot
from sbi.inference import NPE
from sbi.utils import BoxUniform
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_prior,
    process_simulator,
)

from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import Pool


### Simulation Parameters

hgt_rate_max = 1 # Maximum hgt rate
hgt_rate_min = 0 # Minimum hgt rate

theta = 1
rho = 0.1
num_samples = 5

### Define simulator function:

#def simulator_SBI(hgt_rate):
#    return simulator.simulator(theta=theta, rho=rho, num_samples=num_samples, hgt_rate=hgt_rate)

prior = BoxUniform(low=hgt_rate_min * torch.ones(1), high=hgt_rate_max * torch.ones(1))

### Consistency checks:

# Check prior, return PyTorch prior.
prior, num_parameters, prior_returns_numpy = process_prior(prior)

# Check simulator, returns PyTorch simulator able to simulate batches.
simulator_test = process_simulator(simulator.simulator_SBI, prior, prior_returns_numpy)

# Consistency check after making ready for sbi.
# check_sbi_inputs(simulator_test, prior)


### Simulate data:

hgt_rate = prior.sample((10,))
